In [1]:
#Required libraries: Numpy, Pandas, Sklearn, graphviz, numexpr

import numpy as np
import pandas as pd
import graphviz
import numexpr
from subprocess import call

from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix




In [2]:
##Part 1: Takes in fully labelled dataframe and creates dummy columns that are binary.

### data needs to be preformatted in several ways to work: 
    #0) data is of expression values in in a sample X genes matrix with a column for cluster labels   
    #1) The header for the cluster labels MUST BE "Clusters" while the labels themselves must be strings 
        ##(simple numerics will cause a teriminal error)
    #2) The gene IDs/ symbols MUST BE stripped of ".", "-", and "@", or other troublesome characters (I replace with "_")    

dataFull = pd.read_table("/path/to/data/here", index_col = 0) #Data loading
PrecolNum = len(dataFull.columns) #Gives the pre-Dummy columns which is used to bound the main loop

dataDummy = pd.get_dummies(dataFull, columns=["Clusters"], prefix = "", prefix_sep = "") 

#generates a dummy column per cluster, rendering each a binary classifcation (one-vs-all). This will add a column per
#cluster to the end of the data frame. The header is the un-prefixed name. This becomes important in df.loc calls later.

PostcolNum = len(dataDummy.columns) #Gives the post-Dummy columns which is used to bound the main loop

medianValues = dataFull.groupby(by="Clusters").median() #computes the median expression value per cluster for each gene

QuantileValues75 = dataFull.groupby(by="Clusters").quantile(0.25, 0) 

#computes expression value per cluster for each gene where 75% of the cells have expression 


adjustedColumns = PrecolNum-1 
# column calc for main loop bounding adjustment for removed Cluster Column (sacrificed during the creation of dummies)


clusters2Loop=PostcolNum-adjustedColumns #Gives the number of dummy columns which should equal the number of clusters

print clusters2Loop #Tells you the number of clusters so you know we are live and the above went well...


5


In [ ]:
counter = 0
rankedDict =  {}  ###stores the top ten features from RF
f1_store_1D = {} ##Stores all of the f-measure results Key format = clusterName&gene1Name&gene2Name...

for column in dataDummy.columns[PrecolNum-1:PostcolNum]: #main loop, iterates by through dummy columns to perform RF
        if counter == clusters2Loop: #Ends loop when # of columns/clusters is achieved
            break
        print column
        ###Random Forest one-vs-all     
        x_train = dataDummy[list(dataDummy.columns[0:PrecolNum-1])]  #get features (genes)
        names = dataDummy.columns[0:PrecolNum-1]
        y_train =dataDummy[column] # get binary labels
        rf = RandomForestClassifier(n_estimators=100, n_jobs = 1, random_state=123456) #initialize RF, using 1 cores 
        #### FOR LARGER Data sets or HPC environment change n_jobs = to -1 #####
        rf.fit(x_train, y_train) # Run RF
        Ranked_Features=sorted(zip(map(lambda x: round(x, 4), rf.feature_importances_), names),reverse=True) 
        # Get feature rankings from rf       
        
        counter += 1
        
        ####Get top ten list
        RankedList = [] #Used to store features from RF ranked list and test
        midcounter = 0

        for x in Ranked_Features: #this loop builds a list of top ranked features, default set at 15 genes (testing 6 but negatives get removed)
            RankedList.append(x[1])
            rankedDict[column] = RankedList
            if midcounter==9:            
                break
            midcounter +=1
        
        
        ##initialize testArray which will be the workhorse for f-measure calculations below
        testArray = dataDummy[[column]] #Get "true" cluster labels
        testArray.columns = ['y_true'] #Rename y_true for scikitlearn convention
        
        ##This loop iterates through the ranked list removing genes that have a median value of 0 (sensitive to zero-inflation)  
        Positive_RankedList = [] #list for store filtered ranked genes
        for i in RankedList: 
            if medianValues.loc[column, i] >= 0:
                #print medianValues.loc[column, i]
                Positive_RankedList.append(i)
            else:
                print i
                print medianValues.loc[column, i]
                print "Is Right Out!"        
        
        ##First Gene f-measure testing loop. This loop tests the filtered gene list using f-measure 
        #and an expression where 75% of cells have expression. 

        #Best_dict = {} #Local store for results
        Best_list=[] #Stores list of best genes that are move on to the next loop
        ranking_dict={} #used to rank f-measure results and add decide which to append to Best_list[]
        for index, item in enumerate(Positive_RankedList):
            
            testArray['y_pred']= 0 ###adds zero vector, meaning all cells begin as negative prediction
            current_value = QuantileValues75.loc[column,Positive_RankedList[index]] #finds expression value to evaluate at
            print "INPUTS:"
            str1 = Positive_RankedList[index] #Name of gene
            queryString1 = unicode(str1)+'>='+ unicode(current_value) #make query into string
            print queryString1
            queryUltra = unicode(queryString1)
            Ineq1=dataFull.query(queryUltra) # runs query finding cells that meet expression inequality
            testList=Ineq1.index.tolist() #Finds index of cells that passed inequality
            testArray.loc[testList, 'y_pred'] = 1 #uses index of cells found by query to update prediction column from 0 to 1
            f1 = f1_score(testArray['y_true'], testArray['y_pred'], average= 'binary') #Runs f-measure on testArray
            dictName = column+"&"+Positive_RankedList[index] #creates dict key
            f1_store_1D[dictName] = f1 #stores result in dictionary for reporting
            ranking_dict[Positive_RankedList[index]] = f1 #stores gene as key and f-measure as value for ranking and removal from list used for looping
            print "The f-score is:"             
            print f1
          
        Best_list.insert(0,max(ranking_dict, key=ranking_dict.get)) #Uses gene(key) : f-measure values 
        ##to find best gene and adds it to list (this moves to the second loop)
        Positive_RankedList.remove(Best_list[0]) #Removes the best first gene from looping list
        current_value = QuantileValues75.loc[column,Best_list[0]] #Set value for 1st gene for rest of analysis
        str1 = Best_list[0] #Set geneName for 1st gene for rest of analysis
        
        
        ###### The remaining loops take the remaining Positive_RankedList and test all two inequalies using the best result
        ####### from the first gene analysis above. This process is repeated until six inequalities are tested.
        ######## A typical error occurs when Positive_RankedList runs out of genes due to the zero filtering above
        ######### The code will then throw an "empty string" error. One can simply increase the number of genes in RankedList.
        
        
        ###Two genes
        print "Time for two"
        ranking_dict={}
        for index, item in enumerate(Positive_RankedList):
            testArray['y_pred']= 0 ###adds zero vector that will altered in the subroutine#
            current_value_1 = QuantileValues75.loc[column,Positive_RankedList[index]]
            str1_1 = Positive_RankedList[index]
            #print current_value
            queryString1 = unicode(str1)+'>='+ unicode(current_value)+'&'+ unicode(str1_1)+'>='+ unicode(current_value_1)
            print queryString1
            queryUltra = unicode(queryString1)
            Ineq1=dataFull.query(queryUltra)        
            testList=Ineq1.index.tolist()
            testArray.loc[testList, 'y_pred'] = 1
            f1 = f1_score(testArray['y_true'], testArray['y_pred'], average= 'binary') 
            dictName = column+"&"+Best_list[0]+'&'+Positive_RankedList[index]
            f1_store_1D[dictName] = f1
            ranking_dict[Positive_RankedList[index]] = f1
            print "The f-score is:" 
            print f1
            
        Best_list.insert(1,max(ranking_dict, key=ranking_dict.get))
        Positive_RankedList.remove(Best_list[1])
        current_value_1 = QuantileValues75.loc[column,Best_list[1]]
        str1_1 = Best_list[1]
     
        ###Three genes
        print "Time for three"
        ranking_dict={}

        for index, item in enumerate(Positive_RankedList):
            testArray['y_pred']= 0 ###adds zero vector that will altered in the subroutine#
            current_value_2 = QuantileValues75.loc[column,Positive_RankedList[index]]
            str1_2 = Positive_RankedList[index]
            queryString1 = unicode(str1)+'>='+ unicode(current_value)+'&'+ unicode(str1_1)+'>='+ unicode(current_value_1)+'&'+ unicode(str1_2)+'>='+ unicode(current_value_2)
            print queryString1
            queryUltra = unicode(queryString1)
            Ineq1=dataFull.query(queryUltra)        
            testList=Ineq1.index.tolist()
            testArray.loc[testList, 'y_pred'] = 1
            f1 = f1_score(testArray['y_true'], testArray['y_pred'], average= 'binary') 
            dictName = column+"&"+Best_list[0]+"&"+Best_list[1]+'&'+Positive_RankedList[index]
            f1_store_1D[dictName] = f1
            ranking_dict[Positive_RankedList[index]] = f1
            print "The f-score is:" 
            print f1
            
        Best_list.insert(2,max(ranking_dict, key=ranking_dict.get))
        Positive_RankedList.remove(Best_list[2])
        current_value_2 = QuantileValues75.loc[column,Best_list[2]] 
        str1_2 = Best_list[2]
        
        print "Time for four"
        
        ranking_dict={}
        for index, item in enumerate(Positive_RankedList):
            testArray['y_pred']= 0 ###adds zero vector that will altered in the subroutine#
            current_value_3 = QuantileValues75.loc[column,Positive_RankedList[index]]
            print "INPUTS:"
            str1_3 = Positive_RankedList[index]
            print "fourth gene"
            print str1_3            
            print queryString1
            queryUltra = unicode(queryString1)
            Ineq1=dataFull.query(queryUltra)        
            queryString1 = unicode(str1)+'>='+ unicode(current_value)+'&'+ unicode(str1_1)+'>='+ unicode(current_value_1)+'&'+ unicode(str1_2)+'>='+ unicode(current_value_2)+'&'+ unicode(str1_3)+'>='+ unicode(current_value_3)
            testList=Ineq1.index.tolist()
            testArray.loc[testList, 'y_pred'] = 1
            f1 = f1_score(testArray['y_true'], testArray['y_pred'], average= 'binary') 
            dictName = column+"&"+Best_list[0]+"&"+Best_list[1]+"&"+Best_list[2]+'&'+Positive_RankedList[index]
            f1_store_1D[dictName] = f1
            ranking_dict[Positive_RankedList[index]] = f1
            print "The f-score is:"  
            print f1
        Best_list.insert(3,max(ranking_dict, key=ranking_dict.get))
        Positive_RankedList.remove(Best_list[3])
        current_value_3 = QuantileValues75.loc[column,Best_list[3]] 
        str1_3 = Best_list[3]

        ##Time for five
        print "Five Genes"
        ranking_dict={}
        for index, item in enumerate(Positive_RankedList):
            testArray['y_pred']= 0 ###adds zero vector that will altered in the subroutine#
            current_value_4 = QuantileValues75.loc[column,Positive_RankedList[index]]
            str1_4 = Positive_RankedList[index]
            queryString1 = unicode(str1)+'>='+ unicode(current_value)+'&'+ unicode(str1_1)+'>='+ unicode(current_value_1)+'&'+ unicode(str1_2)+'>='+ unicode(current_value_2)+'&'+ unicode(str1_3)+'>='+ unicode(current_value_3)+'&'+ unicode(str1_4)+'>='+ unicode(current_value_4)
            print queryString1
            queryUltra = unicode(queryString1)
            Ineq1=dataFull.query(queryUltra)        
            testList=Ineq1.index.tolist()
            testArray.loc[testList, 'y_pred'] = 1
            f1 = f1_score(testArray['y_true'], testArray['y_pred'], average= 'binary') 
            dictName = column+"&"+Best_list[0]+"&"+Best_list[1]+"&"+Best_list[2]+"&"+Best_list[3]+'&'+Positive_RankedList[index]
            f1_store_1D[dictName] = f1
            ranking_dict[Positive_RankedList[index]] = f1
            print "The f-score is:"  
            print f1
        Best_list.insert(4,max(ranking_dict, key=ranking_dict.get))
        Positive_RankedList.remove(Best_list[4])
        current_value_4 = QuantileValues75.loc[column,Best_list[4]] 
        str1_4 = Best_list[4]
       
    
        ##Time for six
        print "Six genes"
        ranking_dict={}
        for index, item in enumerate(Positive_RankedList):
            testArray['y_pred']= 0 ###adds zero vector that will altered in the subroutine#
            current_value_5 = QuantileValues75.loc[column,Positive_RankedList[index]]
            str1_5 = Positive_RankedList[index]
            queryString1 = unicode(str1)+'>='+ unicode(current_value)+'&'+ unicode(str1_1)+'>='+ unicode(current_value_1)+'&'+ unicode(str1_2)+'>='+ unicode(current_value_2)+'&'+ unicode(str1_3)+'>='+ unicode(current_value_3)+'&'+ unicode(str1_4)+'>='+ unicode(current_value_4)+'&'+ unicode(str1_5)+'>='+ unicode(current_value_5)
            print queryString1
            queryUltra = unicode(queryString1)
            Ineq1=dataFull.query(queryUltra)        
            testList=Ineq1.index.tolist()
            testArray.loc[testList, 'y_pred'] = 1
            f1 = f1_score(testArray['y_true'], testArray['y_pred'], average= 'binary') 
            dictName = column+"&"+Best_list[0]+"&"+Best_list[1]+"&"+Best_list[2]+"&"+Best_list[3]+"&"+Best_list[4]+'&'+Positive_RankedList[index]
            f1_store_1D[dictName] = f1
            ranking_dict[Positive_RankedList[index]] = f1
            print "The f-score is:"  
            print f1
        Best_list.insert(5,max(ranking_dict, key=ranking_dict.get))
        Positive_RankedList.remove(Best_list[5])
        #current_value_5 = QuantileValues75.loc[column,Best_list[5]] 
        #str1_5 = Best_list[5]




In [4]:
#######Final Report generation

f1_store_1D_df=pd.DataFrame() #instantiates dataframe
f1_store_1D_df=pd.DataFrame.from_dict(f1_store_1D, orient='index', dtype=str) #converts dict store of all results to dataframe
f1_store_1D_df.columns = ["f-measure"] #Renames column to f-measure
f1_store_1D_df['markerCount'] = f1_store_1D_df.index.str.count('&') #counts number of markers using "&" delimiters
f1_store_1D_df.reset_index(level=f1_store_1D_df.index.names, inplace=True) #make index a column
f1_store_1D_df_MC= f1_store_1D_df['index'].apply(lambda x: pd.Series(x.split('&'))) #parses former index into many columns using "&" delimiters

NSForest_Results_Table=f1_store_1D_df.join(f1_store_1D_df_MC) #joins two dataframes (original and parsed values)
NSForest_Results_Table.rename(columns={0:'clusterName'},inplace=True) #rename column by position
NSForest_Results_Table.sort_values(by=['clusterName','f-measure','markerCount'],ascending= [True, False, True], inplace = True) #Sorts by columns given ascending criteria
NSForest_Results_Table.to_csv('NSForest_Results_Table.csv') #Saves full results to root directory, add path if desired.